In [27]:
#pip install citipy - citypy already installed

In [54]:
# Import dependencies.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# Add citypy lbrary
from citipy import citipy

# Add requests module.
import requests

# Import the API key from the config.py file.
from config import weather_api_key

# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [55]:
# Generate Lats range from -90 to 90
# Generate Lngs range from -180 to 180

In [56]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=10)
lngs = np.random.uniform(low=-180.000, high=180.000, size=10)

# Zip method combines the lats and lngs into a list of pairs. 
lat_lngs = zip(lats, lngs)
lat_lngs

In [57]:
# Add latitudes and longitudes to a list.
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [58]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

9

In [59]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
# print(url)

In [63]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    #print(requests.get(city_url).json())
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | yellowknife
Processing Record 2 of Set 1 | castro
Processing Record 3 of Set 1 | rikitea
Processing Record 4 of Set 1 | nuuk
Processing Record 5 of Set 1 | hualmay
Processing Record 6 of Set 1 | alofi
Processing Record 7 of Set 1 | severo-kurilsk
Processing Record 8 of Set 1 | bredasdorp
Processing Record 9 of Set 1 | hobart
-----------------------------
Data Retrieval Complete      
-----------------------------


In [67]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Yellowknife,62.4560,-114.3525,-6.59,96,100,4.61,CA,2022-01-29 17:53:10
1,Castro,-24.7911,-50.0119,59.76,98,100,5.93,BR,2022-01-29 17:53:11
2,Rikitea,-23.1203,-134.9692,77.23,75,89,12.84,PF,2022-01-29 17:53:11
3,Nuuk,64.1835,-51.7216,13.87,85,100,17.00,GL,2022-01-29 17:50:10
4,Hualmay,-11.0964,-77.6139,75.43,70,89,11.43,PE,2022-01-29 17:48:57
5,Alofi,-19.0595,-169.9187,73.29,100,80,3.44,NU,2022-01-29 17:53:12
6,Severo-Kurilsk,50.6789,156.1250,23.34,70,98,19.77,RU,2022-01-29 17:53:12
7,Bredasdorp,-34.5322,20.0403,69.28,68,0,16.11,ZA,2022-01-29 17:49:57
8,Hobart,-42.8794,147.3294,60.66,76,100,6.91,AU,2022-01-29 17:53:13


In [70]:
# Reorder the DataFrame.
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]

# View city_data_df
city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Yellowknife,CA,2022-01-29 17:53:10,62.4560,-114.3525,-6.59,96,100,4.61
1,Castro,BR,2022-01-29 17:53:11,-24.7911,-50.0119,59.76,98,100,5.93
2,Rikitea,PF,2022-01-29 17:53:11,-23.1203,-134.9692,77.23,75,89,12.84
3,Nuuk,GL,2022-01-29 17:50:10,64.1835,-51.7216,13.87,85,100,17.00
4,Hualmay,PE,2022-01-29 17:48:57,-11.0964,-77.6139,75.43,70,89,11.43
5,Alofi,NU,2022-01-29 17:53:12,-19.0595,-169.9187,73.29,100,80,3.44
6,Severo-Kurilsk,RU,2022-01-29 17:53:12,50.6789,156.1250,23.34,70,98,19.77
7,Bredasdorp,ZA,2022-01-29 17:49:57,-34.5322,20.0403,69.28,68,0,16.11
8,Hobart,AU,2022-01-29 17:53:13,-42.8794,147.3294,60.66,76,100,6.91


In [71]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")